# End-to-End Reconstruction Tutorial

This tutorial walks through a complete multi-view stereo reconstruction workflow using the AquaMVS Python API. We'll start from synchronized multi-camera images, run the reconstruction pipeline, and produce a 3D surface mesh.

By the end of this tutorial, you'll have:
- Loaded and inspected a pipeline configuration
- Executed the reconstruction pipeline
- Examined intermediate outputs (depth maps, consistency maps)
- Visualized the fused point cloud
- Exported the final mesh to various formats

## Example Dataset

This tutorial assumes you have downloaded the example dataset. If you haven't already:

1. Download from: [https://github.com/tlancaster6/AquaMVS/releases/download/v0.1.0-example-data/aquamvs-example-dataset.zip](https://github.com/tlancaster6/AquaMVS/releases/download/v0.1.0-example-data/aquamvs-example-dataset.zip)
2. Extract to a local directory
3. The dataset includes:
   - Synchronized camera images (or videos)
   - AquaCal calibration JSON
   - Pre-configured `config.yaml`

## Prerequisites

Ensure you have installed AquaMVS and its dependencies:

```bash
# Install PyTorch (CPU or CUDA version depending on your hardware)
pip install torch torchvision --index-url https://download.pytorch.org/whl/cu121

# Install AquaMVS
pip install aquamvs

# Install optional dependencies (LightGlue, RoMa)
pip install git+https://github.com/cvg/LightGlue.git@edb2b83
pip install git+https://github.com/tlancaster6/RoMaV2.git
```

See the [Installation Guide](../installation.rst) for detailed instructions.

## Setup and Imports

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np

from aquamvs import Pipeline, PipelineConfig

# Point to example dataset (adjust path as needed)
DATA_DIR = Path("example_data")
CONFIG_PATH = DATA_DIR / "config.yaml"

# Verify the config exists
if not CONFIG_PATH.exists():
    raise FileNotFoundError(
        f"Config file not found at {CONFIG_PATH}. "
        "Download the example dataset or adjust DATA_DIR."
    )

## 1. Load and Inspect Configuration

The pipeline configuration defines all parameters for reconstruction: camera paths, calibration, feature matching settings, depth estimation parameters, and output options.

In [ ]:
# Load configuration from YAML
config = PipelineConfig.from_yaml(CONFIG_PATH)

# Inspect key parameters
print(f"Cameras: {list(config.camera_video_map.keys())}")
print(f"Output directory: {config.output_dir}")
print(f"Matcher type: {config.sparse_matching.matcher_type}")
print(f"Pipeline mode: {config.reconstruction.pipeline_mode}")
print(f"Device: {config.runtime.device}")

**Expected output:** List of camera names (e.g., `['e3v82e0', 'e3v82e1', ...]`), output directory path, matcher type (`'lightglue'` or `'roma'`), pipeline mode (`'full'` or `'sparse'`), and device (`'cpu'` or `'cuda'`).

## 2. Run the Pipeline

The `Pipeline` class provides the primary programmatic interface. Calling `.run()` executes the full reconstruction workflow:

1. **Undistortion**: Apply camera calibration to remove lens distortion
2. **Feature Matching**: Extract and match features across camera pairs (LightGlue or RoMa)
3. **Triangulation**: Compute 3D points from feature correspondences (sparse mode) or...
4. **Plane Sweep Stereo**: Dense depth estimation via photometric cost volume (full mode)
5. **Depth Fusion**: Merge multi-view depth maps into a single point cloud
6. **Surface Reconstruction**: Generate a triangle mesh from the point cloud

This may take several minutes depending on image resolution, number of cameras, and hardware.

In [ ]:
# Initialize pipeline
pipeline = Pipeline(config)

# Run reconstruction
# This will process all frames according to config.preprocessing settings
pipeline.run()

**Expected output:** Progress bars and log messages indicating stage completion. Final message: `"Pipeline complete"`.

## 3. Examine Intermediate Results

The pipeline saves intermediate outputs to the output directory, organized by frame. Let's load and visualize depth maps and consistency maps for frame 0.

In [ ]:
# Path to frame 0 output
output = Path(config.output_dir) / "frame_000000"

# Pick the first camera to visualize
camera_names = list(config.camera_video_map.keys())
cam = camera_names[0]

print(f"Visualizing outputs for camera: {cam}")

### Depth Map

Depth maps represent the distance along each ray from the camera to the water surface. Values are in meters (ray depth, not world Z).

In [ ]:
# Load depth map (saved as NPZ with 'depth' array)
depth_data = np.load(output / f"{cam}_depth.npz")
depth = depth_data["depth"]

# Visualize
plt.figure(figsize=(12, 8))
plt.imshow(depth, cmap="viridis")
plt.colorbar(label="Depth (m)", shrink=0.8)
plt.title(f"Depth Map — {cam}")
plt.axis("off")
plt.tight_layout()
plt.show()

# Print statistics
valid_mask = ~np.isnan(depth)
print(f"Depth range: {np.nanmin(depth):.3f} to {np.nanmax(depth):.3f} m")
print(
    f"Valid pixels: {valid_mask.sum()} / {depth.size} ({100 * valid_mask.sum() / depth.size:.1f}%)"
)

### Consistency Map

Consistency maps indicate how many source cameras agree with the reference camera's depth estimate at each pixel. Higher values (warmer colors) indicate more reliable depth.

In [ ]:
# Load consistency map
consistency_data = np.load(output / f"{cam}_consistency.npz")
consistency = consistency_data["consistency"]

# Visualize
plt.figure(figsize=(12, 8))
plt.imshow(consistency, cmap="viridis")
plt.colorbar(label="Consistent views", shrink=0.8)
plt.title(f"Consistency Map — {cam}")
plt.axis("off")
plt.tight_layout()
plt.show()

# Print statistics
print(f"Consistency range: {consistency.min():.0f} to {consistency.max():.0f} views")
print(f"Mean consistency: {consistency[valid_mask].mean():.1f} views")

## 4. Visualize the Fused Point Cloud

The fusion stage merges all camera depth maps into a single 3D point cloud, saved as `fused.ply`.

In [ ]:
import open3d as o3d

# Load fused point cloud
pcd_path = output / "fused.ply"
pcd = o3d.io.read_point_cloud(str(pcd_path))

print(f"Point cloud: {len(pcd.points)} points")
print(f"Has colors: {pcd.has_colors()}")
print(f"Has normals: {pcd.has_normals()}")

# Compute bounds
bbox = pcd.get_axis_aligned_bounding_box()
print(f"Bounding box: {bbox.get_extent()} m")

**Note on visualization:** `o3d.visualization.draw_geometries(pcd)` requires a display (X server on Linux, windowed environment). In Jupyter, you can:

- Use Open3D's Jupyter visualizer: `o3d.visualization.draw(pcd)` (requires `open3d>=0.16`)
- Export to HTML for web-based viewing
- Save a rendered image for display

For headless environments, skip interactive visualization and proceed to mesh export.

## 5. Surface Reconstruction and Export

The surface reconstruction stage converts the point cloud into a triangle mesh. The default method is Poisson reconstruction, which produces a watertight mesh.

In [ ]:
# Load reconstructed mesh
mesh_path = output / "surface.ply"
mesh = o3d.io.read_triangle_mesh(str(mesh_path))

print(f"Mesh: {len(mesh.vertices)} vertices, {len(mesh.triangles)} triangles")
print(f"Has vertex colors: {mesh.has_vertex_colors()}")
print(f"Has vertex normals: {mesh.has_vertex_normals()}")

### Export to Other Formats

AquaMVS provides an `export_mesh` function to convert meshes to OBJ, STL, GLTF, or GLB formats with optional simplification.

In [ ]:
from aquamvs import export_mesh

# Export to OBJ (widely supported, preserves colors)
obj_path = output / "surface.obj"
export_mesh(str(mesh_path), str(obj_path))
print(f"Exported to OBJ: {obj_path}")

# Export to STL with simplification (for 3D printing)
stl_path = output / "surface_simplified.stl"
export_mesh(str(mesh_path), str(stl_path), simplify=10000)
print(f"Exported simplified mesh to STL: {stl_path}")

# Export to GLB (compact, web-ready)
glb_path = output / "surface.glb"
export_mesh(str(mesh_path), str(glb_path))
print(f"Exported to GLB: {glb_path}")

## Next Steps

Now that you've completed a basic reconstruction, explore:

- **[CLI Guide](../cli_guide.md)**: Command-line workflow for batch processing
- **[Theory](../theory/index.rst)**: Understand the refractive geometry and algorithms
- **[API Reference](../api/index.rst)**: Detailed documentation of all modules and functions

### Configuration Tips

- **Switch matchers**: Set `sparse_matching.matcher_type: "roma"` for dense matching (slower, more accurate)
- **Adjust depth range**: Modify `reconstruction.depth_min` and `depth_max` to focus on your region of interest
- **GPU acceleration**: Set `runtime.device: "cuda"` if you have a CUDA-capable GPU
- **Quality vs. speed**: Increase `reconstruction.num_depth_hypotheses` (default: 64) for higher quality at the cost of longer runtime

### Multi-Frame Reconstruction

To process multiple frames, adjust `preprocessing` settings in the config:

```yaml
preprocessing:
  frame_start: 0
  frame_stop: 100  # Process frames 0-99
  frame_step: 10   # Every 10th frame
```

Each frame's outputs will be saved to `output/frame_XXXXXX/`.